<a href="https://colab.research.google.com/github/sheetalkbc/MyColab-Notebooks/blob/master/Transformer_%26_Autoencoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%tensorflow_version 2.x

In [ ]:
!pip install -U sentence-transformers

In [ ]:
!pip install -q transformers

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from transformers import FeatureExtractionPipeline,AutoTokenizer,AutoModel,Pipeline
import transformers
import tqdm

#from sklearn.model_selection import train_test_split

import tensorflow as tf

In [ ]:
#from google.colab import files
#import io

#uploaded = files.upload()

#df = pd.read_csv(io.StringIO(uploaded['TruckRoll.csv'].decode('utf-8')))

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/sheetalkbc/TestDataFiles/main/TruckRoll.csv?token=AJU6QFGGJUHYDPCKXFHHQUTA52EO4')

In [ ]:
df.drop('Unnamed: 0',inplace=True,axis=1)

In [ ]:
F_model = AutoModel.from_pretrained("xlm-roberta-large-finetuned-conll03-german",output_hidden_states=True)
F_Tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-large-finetuned-conll03-german",use_fast=True,max_len=512)

pipe = FeatureExtractionPipeline(F_model,F_Tokenizer,device=0)

In [ ]:
tf.config.list_physical_devices('GPU')

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('sentence-transformers/stsb-xlm-r-multilingual')


In [ ]:
df_nonConfigIssue = df.text.loc[df.labels==0]
df_ConfigIssue = df.text.loc[df.labels==0]

In [ ]:
batch_size = 64
features = np.empty((0,768), float)

for i in tqdm.tqdm(range(0, df_nonConfigIssue.shape[0], batch_size)):
    batch = df_nonConfigIssue.tolist()[i:i+batch_size]
    feature_list = model.encode(batch)
    Arr = np.array(feature_list)
    features = np.append(features,Arr,axis=0)

np.shape(features)  

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
AutoEncoderModel = keras.Sequential(
    [
      layers.InputLayer(input_shape=(768,)),
        layers.Dense(768,activation="relu"),
        layers.Dense(700,activation="relu"),
        layers.Dense(350,activation="relu"),
        layers.Dense(700,activation="relu"),
        layers.Dense(768,activation="relu"),
    ]
)
AutoEncoderModel.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), loss="mse")
AutoEncoderModel.summary()

In [ ]:
history = AutoEncoderModel.fit(
    features,
    features,
    epochs=50,
    batch_size=128,
    validation_split=0.1,
    callbacks=[
        keras.callbacks.EarlyStopping(monitor="val_loss", patience=5, mode="min")
    ],
)

In [ ]:
plt.plot(history.history["loss"], label="Training Loss")
plt.plot(history.history["val_loss"], label="Validation Loss")
plt.legend()
plt.show()

In [ ]:
# Get train MAE loss.
x_train_pred = AutoEncoderModel.predict(features)
train_mae_loss = np.mean(np.abs(x_train_pred - features), axis=1)

plt.hist(train_mae_loss, bins=50)
plt.xlabel("Train MAE loss")
plt.ylabel("No of samples")
plt.show()

# Get reconstruction loss threshold.
threshold = np.max(train_mae_loss)
print("Reconstruction error threshold: ", threshold)

In [ ]:
anomalies = train_mae_loss > threshold
np.where(anomalies)

**Testing**

In [ ]:
Config_features = model.encode(df_ConfigIssue.tolist())

x_test_pred = AutoEncoderModel.predict(Config_features)
test_mae_loss = np.mean(np.abs(x_test_pred - Config_features), axis=1)

In [ ]:
plt.hist(test_mae_loss, bins=50)
plt.xlabel("Train MAE loss")
plt.ylabel("No of samples")
plt.show()

# Get reconstruction loss threshold.
threshold = np.max(test_mae_loss)
print("Reconstruction error threshold: ", threshold)